# Single Layer Perceptron

We have now talked about the basics of (artificial) Neural Networks (NN, or ANN), and even discussed some of the history and fancier applications. 

In this exercise, we will focus on the single-layer perceptron. Significant credit is due to [Qingkai Kong](http://qingkaikong.blogspot.com/2016/11/machine-learning-3-artificial-neural.html) from Berkeley, and [iamtrask](http://iamtrask.github.io/2015/07/12/basic-python-network/).

## Dataset to be evaluated with a perceptron algorithm

Let's use an example of a very simple dataset.

We have 5 input data, each of which have 3 features. 

The "true" values (or "targets") for each of these 5 data are simply 0 or 1. This is very typical of a classification problem, as we discussed in class. 


| Data   |Feature 1|Feature 2|Feature 3|Target|
|:------:|:------:|:------:|:------:|:----:|
| Event 1|    0   |    0   |    1   |   0  |
| Event 2|    1   |    1   |    1   |   1  |
| Event 3|    1   |    0   |    1   |   1  |
| Event 4|    0   |    1   |    1   |   0  |
| Event 5|    0   |    1   |    0   |   1  |




----
# DISCUSSION
1. Can you figure out any patterns by eye?
1. Do you see any correlations?
1. Could you think of an analytic algorithm that would predict these outputs?
----

## Goal and structure of the algorithm

We want to build and train a simple Perceptron model that can output the correct target by feeding into the 3 features from the data samples. See the following table. 

Let's have another look of the structure of our model:

<img src="https://raw.githubusercontent.com/UChicagoPhysics/PHYS250/master/Slides/Lecture16/singlelayer_perceptron_3.png" width="600"/>  

We have two layers in this model: the *input* layer and the *output* layer. 

For the input layer, it has 3 features that connect to the output layer via 3 weights. Also we added one more node with 1 as input, and the weight associated with it is the $w_0$, which serves as a bias.  

----
# DISCUSSION
1. Why did we add this bias term? What does the bias term achieve?
----

## Steps to training the perceptron

The steps we will take to train this model is:

1. Initialize the weights to small random numbers with both positive or negative values.
2. Calculate the output: $y_j = f(\vec{w}\cdot\vec{x}_j)$
3. Determine the error: $\epsilon_j = d_j - y_j$
4. Update the weights: $w^{\mathrm{new}}_i = w_i + r ( \epsilon_j \cdot \vec{x}_j )_{i}$

### A few details

#### Bias term
As suggested to you to think about above, the bias term is important, but a bit of a detail. 

Think about the case if all our features are 0, and no matter what weights we have, we will always have 0 to the output layer. Therefore, adding this 1 extra node can avoid this problem. It is usually called bias term.   

#### Activation function   
Since the output layer only has one node, that sum all the passed input and determine whether the output should fire or not. 

Usually we use `1` indicate fire and `0` indicate not fire. 

In this case, we can see the sum of all the input via the weights are: $z = w_0 + x_1 * w_1 + x_2 * w_2 + x_3 * w_3$. 

But what we want as the output is either 0 or 1 to represent two classes, or a number between 0 and 1, so that we can use it as a probability. Since this number $z$ can be anything, how can we scale it to a value within 0 to 1, as a probability, or simply 0 and 1? 

Here, the [activation function](https://en.wikipedia.org/wiki/Activation_function) comes to help. The purpose of using the activation function is to  achieve this goal. Therefore, we can choose many different activation function, but the most commonly used activation function is the sigmoid (also called logistic) function, which is shown below, to scale the value $z$ to a number between 0 and 1 to represent probability. The advantage of using this function lies in its ability to get the derivative. Since the derivative will be used to update the weights during learning, this property from the sigmoid function is very desirable. The horizontal axis is the z value from the above, it can be any value from negative to positive. The vertical axis is the scaled value from 0 to 1 that we can use as our output. We can see that, if the z value we get is a large positive number, say 10, or a very small negative number, i.e. -10, the scaled value will be either 1 or 0. We can say at these cases, the network is very confident that the class is belong to certain class. But if we have z relatively close to 0, for example, -1 to 1, then the scaled value will be around 0.2 to 0.7, which also indicates the network is not so confident about the result. 

<img src="https://raw.githubusercontent.com/UChicagoPhysics/PHYS250/master/Slides/Lecture16/sigmoid-derivative.png" width="600"/>   

Therefore, by input the features to the network, and we can get the perceptron network to classify an object into two different class. But what if the result is wrong by input the features? This is totally possible, since we initialize the weights as random small numbers. To solve this problem, we need the perceptron to have the ability to learn from the data. 

### How to learn from error  

Learning will be achieved by first estimate how much error we make by using current weights, and then find some way to update the weights that we can reduce the error in the next time. I won't say too much here, but we will see more details with the explanation of the following code.  

### Learning rate   

Usually, we also need a learning rate to control how fast the network learns. We will see later in the code that the learning rate decide how much to change the weight by. We could miss it out, which would be the same as setting it to 1. But if we do that, the weights change a lot whenever there is a wrong answer, which tends to make the network unstable, so that it never settles down. The cost of having a small learning rate is that the weights need to see the inputs more often before they change significantly, so that the network takes longer to learn. However, it will be more stable and resistant to noise (errors) and inaccuracies in the data. 

Let's look at the code to do the above things to learn from data, and explain them line by line below:  

## Build the perceptron

First we need the activation function itself:

In [5]:
import numpy as np

def sigmoid(x,deriv=False):
    sig = 1/(1+np.exp(-x))
    if(deriv==True):
        return sig*(1-sig)
    return sig

Here we define our learning rate, this will control how fast the network learns from the data. Usually this learning rate will be a number within 0 - 1, with 0 means the network will not learn at all, and 1 means the network will learn a full speed. 

In [6]:
learning_rate = 0.4

Next, let's initialize the input dataset as numpy matrix. 

Each row is a single data sample, and each column corresponds to one features (one of the input nodes). 

And we also add the bias term 1. 

You can see that we now have 4 input nodes and 5 training examples.

In [7]:
# input dataset, note we add bias 1 to the input data
X = np.array([[0,0,1],[1,1,1],[1,0,1],[0,1,1],[0,1,0]])
X = np.concatenate((np.ones((len(X), 1)), X), axis = 1)

This initializes our output dataset. ".T" is the transpose function, which to convert our output data to a column vector. You can see that we have 5 rows and 1 column, corresponds to 5 data samples and 1 output node. 

In [8]:
# output dataset           
y = np.array([[0,1,1,0,1]]).T

Before we generate the random weights, we use a seed to make sure that every time we have the same set of random number generated. This is very useful when we test the algorithm, and compare the results with others. This means that your results and my results should be the same. But in reality when you use the algorithm, you don't need to seed it.   

In [9]:
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)

This initializes our weights to connect the input layer to the output layer. Since we have 4 input nodes (including the bias term), we initialize the random weights as dimension (4,1). Also note that the random numbers we initialized are within -1 to 1, with a mean of 0. There is quite a bit of theory that goes into weight initialization. For now, just take it as a best practice that it's a good idea to have a mean of zero in weight initialization. 

In [10]:
# initialize weights randomly with mean 0
weights_0 = 2*np.random.random((4,1)) - 1

We begin the training and make the Perceptron to learn in 50000 iterations. In each of the iteration, the algorithm will learn from the error it made.

In [11]:
# train the network with 50000 iterations
for iter in range(50000):

    # forward propagation
    layer_0 = X
    layer_1_output = sigmoid(np.dot(layer_0,weights_0))

    # how much difference? This will be the error of 
    # our estimation and the true value
    layer1_error = y - layer_1_output

    # multiply how much we missed by the
    # slope of the sigmoid at the values at output layer
    # we also multiply the input to take care of the negative case
    layer1_delta = learning_rate * layer1_error * sigmoid(layer_1_output,True)
    layer1_delta = np.dot(layer_0.T,layer1_delta)

    # update weights by simply adding the delta
    weights_0 += layer1_delta

Note that we explicitly assign out input data to layer_0 (since we have 2 layers, we will call it layer_0, and layer_1). We're going to process all the data at the same time in this implementation, this is called 'batch processing'. There's another type of learning called ['online learning'](https://en.wikipedia.org/wiki/Online_machine_learning), which essentially change the weights whenever there's new data sample available, but we won't talk it here.  

This is the forward propagation. It has two steps, first, the input data from the input layer and propagate to the output layer via the weights, and second, the activation function convert it into a number between 0 and 1. The first step is achieved by the `np.dot` function, `np.dot(layer_0,weights_0)` = $1*w_0 + x_1*w_1 + x_2*w_2 + x_3*w_3$, we can call the derived number is z. The second step is passing this number z to the sigmoid function to get a number between 0 to 1.   

```python
    layer_1_output = sigmoid(np.dot(layer_0,weights_0))
```

In [12]:
print("Output After Training:")
print(layer_1_output)
print('Target')
print(y)

Output After Training:
[[1.46351277e-04]
 [9.99906123e-01]
 [9.99577840e-01]
 [6.58013302e-04]
 [9.99248929e-01]]
Target
[[0]
 [1]
 [1]
 [0]
 [1]]


## Further explanations

This is the error we made using the current weight, it is simply the true answer (y) minus our estimation (layer_1_output). The error of each of the data sample we input into the network is stored here as a 5 by 1 matrix.  

```python
    # how much difference? This will be the error of 
    # our estimation and the true value
    layer1_error = y - layer_1_output
```

This is the most important part - learning. This is the line that makes our algorithm learn from the data. It calculates how much we will change each of the weights in the current iteration. It has 3 parts that multiplied together, (1) the learning rate, (2) the error we just got, and (3) the derivative of the sigmoid function at the output value. 

```python
    # multiply how much we missed by the
    # slope of the sigmoid at the values at output layer
    # we also multiply the input to take care of the negative case
    layer1_delta = learning_rate * layer1_error * sigmoid(layer_1_output,True)
```

The first term is the learning rate, which controls how fast we will learn, it is just a constant in this case, say, we can choose it as 0.4. The larger the values is, the faster the network will learn, but may make the learning unstable and cause the results oscillating that never get the correct results.      

The second term is the error we made. Let's think this error a little more: we have two class, class 0 and class 1. Therefore, we can have two type of error, true class is 0, but our estimation is larger than 0, the other case is true class is 1, but our estimation is less than 1. The error will be a negative number for the first case, but a positive number for the second. If we assume all the input node is positive, and we get an error as negative number, then we want to reduce the weight to make next estimation smaller. Same thing for the other case, if we have the error as a positive number, then we will want to increase the weights, so that in the next iteration, our estimation will be larger. This is the purpose we include the error term here to determine how we change the weights. But wait, what if the input nodes contain negative input? This will switch the values over, and reverse the direction we want to change the weights. How can we avoid this case? Well, it is quite simple, we can just multiply the input into this function, the negative error with negative input multiply will be a positive number, which satisfy our need. This is where line 42 comes in, we separate these two lines for more clear explanation.    

The final term is the derivative of the sigmoid function evaluated at the output value. This derivative is plotted above and included again here:

<img src="https://raw.githubusercontent.com/UChicagoPhysics/PHYS250/master/Slides/Lecture16/sigmoid-derivative.png" width="600"/>   

We can clearly see the slope is different at various values of x, ranging from 0 (when x value is approaching to two end, i.e. either really large or really small, you can also see the slope is very flat) to 1 (when x value is close to the 0, the slope is very steep). This means we will multiply another value that is between 0 to 1 to the error we made that already scaled with the learning rate. 

If we look carefully with the figure below of just the sigmoid, the slope of the green and purple dots are flat than the blue dot, which means they will have a smaller value. The green and purple dots represent more confidence of the result to be one class, for example, the green dot the sigmoid value is about 0.9, which is much close to the class 1. But the blue dot has a sigmoid value 0.5, which we have no idea whether it should belong to class 1 or class 0. Therefore, this slope is large when we have no confidence of determine the class of result, and small when we have more confidence of the class of the results. Now, this makes sense, if the current weights already made a very good estimate of the class (for the green dot), we will want to update the weights really small or even not update it. But if the current estimation is not clear, for example, the blue case, we will want to change the weights relatively large to make it clearer. Therefore, combining the 3 terms, we can get a delta value for each of the weight we want to change in the direction to reduce the error.    

<img src="https://raw.githubusercontent.com/qingkaikong/blog/master/39_ANN_part2_step_by_step/figures/figure3_sigmoid_derivative.jpg" width="600"/> 

As we talked about the negative input case (see line 39 second term explanation), where we need multiply the input to keep the sign correctly (to keep the direction we want to change the weights correctly!).   

```python
    layer1_delta = np.dot(layer_0.T,layer1_delta)
```

This is the last line, it aims to update the current weights by simply adding the delta we just determined to the current weights.  

```python
    # update weights by simply adding the delta
    weights_0 += layer1_delta
```

This line just print out the final results we get after the network learned 50000 iterations, and we can see we get a pretty decent results.   

```python
print(layer_1_output)
```

This concludes this part by showing you all the important part of the ANN (perceptron) algorithm. But the perceptron has its own limation, which actually caused the ANN winter we talked about in the 1970s until researchers found the way to solve it - the multilayer perceptron algorithm, which we will talk in the next part. 